<a href="https://colab.research.google.com/github/dlargod/Swalokal/blob/main/ModellingSwalokal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [33]:
import os
from os import getcwd

import tensorflow_hub as hub
import tensorflow as tf

from tqdm import tqdm

import shutil

print("\u2022 Using TensorFlow Version:", tf.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.12.0
• GPU Device Not Found. Running on CPU


In [34]:
print(tf.__version__)

2.12.0


### Import MobileNetV2 as our Transfered Model

In [35]:
module_selection = ("mobilenet_v2", 224, 1280)  # @param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4".format(
    handle_base
)
IMAGE_SIZE = (pixels, pixels)
print(
    "Using {} with input size {} and output dimension {}".format(
        MODULE_HANDLE, IMAGE_SIZE, FV_SIZE
    )
)

Using https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4 with input size (224, 224) and output dimension 1280



### Import Dataset from Gdrive

In [36]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
path = os.path.join(os.getcwd(), "gdrive", "My Drive")
datapath = os.path.join(path, "Capstone")
train_path = os.path.join(datapath, "Train")
test_path = os.path.join(datapath, "Test")

In [38]:
train_path

'/content/gdrive/My Drive/Capstone/Train'

### Data Augmentation for train dataset

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 20
IMG_SIZE = (200, 200)

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255.)

train_dataset = train_datagen.flow_from_directory(
    train_path, shuffle=True, batch_size=BATCH_SIZE, target_size=IMG_SIZE
)

test_dataset = test_datagen.flow_from_directory(
    test_path, shuffle=True, batch_size=BATCH_SIZE, target_size=IMG_SIZE
)

Found 333 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


### Modelling

In [40]:
do_fine_tuning = False  # @param {type:"boolean"}

In [41]:
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"
IMAGE_SIZE = (224, 224)
FV_SIZE = 1280

feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=IMAGE_SIZE + (3,),
    output_shape=[FV_SIZE],
    trainable=do_fine_tuning,
)

In [42]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax")
])

### (Optional) Unfreeze some layers

In [43]:
NUM_LAYERS = 10  # @param {type:"slider", min:1, max:50, step:1}

if do_fine_tuning:
    feature_extractor.trainable = True

    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

### Model Fitting

In [44]:
if do_fine_tuning:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"],
    )
else:
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

In [45]:
EPOCHS = 30

hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)

Epoch 1/30
17/17 [==============================] - 41s 2s/step - loss: 0.7343 - accuracy: 0.7868 - val_loss: 0.4378 - val_accuracy: 0.9000
Epoch 2/30
17/17 [==============================] - 33s 2s/step - loss: 0.1211 - accuracy: 0.9610 - val_loss: 0.4237 - val_accuracy: 0.8833
Epoch 3/30
17/17 [==============================] - 32s 2s/step - loss: 0.0717 - accuracy: 0.9790 - val_loss: 0.4656 - val_accuracy: 0.8667
Epoch 4/30
17/17 [==============================] - 33s 2s/step - loss: 0.0939 - accuracy: 0.9760 - val_loss: 0.2219 - val_accuracy: 0.9333
Epoch 5/30
17/17 [==============================] - 33s 2s/step - loss: 0.0810 - accuracy: 0.9760 - val_loss: 0.3498 - val_accuracy: 0.8833
Epoch 6/30
17/17 [==============================] - 38s 2s/step - loss: 0.0374 - accuracy: 0.9820 - val_loss: 0.3811 - val_accuracy: 0.8667
Epoch 7/30
17/17 [==============================] - 33s 2s/step - loss: 0.0388 - accuracy: 0.9850 - val_loss: 0.6150 - val_accuracy: 0.8500
Epoch 8/30
17/17 [==

### Save and Export the Model

In [46]:
SWALOKAL_SAVED_MODEL = "swalokal_saved_model"


In [47]:
tf.saved_model.save(model, SWALOKAL_SAVED_MODEL)

In [48]:
%%bash -s $SWALOKAL_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_3_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_3_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_2'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2023-08-01 04:56:17.209944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [49]:
loaded = tf.saved_model.load(SWALOKAL_SAVED_MODEL)

In [50]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_3_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_3_input')})
{'dense_2': TensorSpec(shape=(None, 3), dtype=tf.float32, name='dense_2')}


### Convert The Model using TFLite's Converter

In [51]:
converter = tf.lite.TFLiteConverter.from_saved_model(SWALOKAL_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [52]:
tflite_model_file = "converted_model.tflite"

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

### Save model into .pb

In [53]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.9956821  0.00131544 0.00300239]]



### Zip the whole model

In [54]:
shutil.make_archive("swalokal_saved_model", "zip", "swalokal_saved_model")

'/content/swalokal_saved_model.zip'